# Timesfm Monthly

In [1]:
!pip install timesfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 831.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of googleapis-common-protos to determine which version is compatible with other requirements. This could take a while.
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 24.3 MB/s e

In [1]:
import pandas as pd
import timesfm

2024-09-12 04:53:41.023008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [2]:
data = pd.read_excel("monthly_data.xlsx")

In [3]:
data.head()

,date,warehouse,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,2020-12-01,Prague_1,190208,Christmas Eve2nd Christmas Day,2,0,0,1,0,0,0,0,0,8.3,0.0,45470,839751,Prague_1_2020-12-05Prague_1_2020-12-06Prague_1...
1,2021-01-01,Prague_1,235089,New Years Day,1,0,0,1,0,0,0,0,0,166.3,112.0,53790,996212,Prague_1_2021-01-01Prague_1_2021-01-02Prague_1...
2,2021-02-01,Prague_1,222102,0,0,0,0,0,0,0,0,0,0,37.0,134.0,49832,943849,Prague_1_2021-02-01Prague_1_2021-02-02Prague_1...
3,2021-03-01,Prague_1,254258,International womens day,0,0,0,0,14,0,0,0,0,24.9,3.0,55508,1094886,Prague_1_2021-03-01Prague_1_2021-03-02Prague_1...
4,2021-04-01,Prague_1,242598,Good FridayEaster Monday,4,0,0,1,0,0,0,0,0,13.4,2.0,50988,1041067,Prague_1_2021-04-01Prague_1_2021-04-02Prague_1...


In [4]:
data = data.drop(["holiday_name", 'shutdown', 'mini_shutdown', 'blackout', 'frankfurt_shutdown', "id", 'blackout', 'mov_change', 'frankfurt_shutdown','precipitation', 'snow', 'user_activity_1', 'user_activity_2'], axis=1)

In [5]:
#rename as per requirement of the forecast libraries
data = data.rename(columns={"warehouse": "unique_id", "date": "ds", "orders": "y"})

In [6]:
data.head()

,ds,unique_id,y,holiday,shops_closed,winter_school_holidays,school_holidays
0,2020-12-01,Prague_1,190208,2,1,0,0
1,2021-01-01,Prague_1,235089,1,1,0,0
2,2021-02-01,Prague_1,222102,0,0,0,0
3,2021-03-01,Prague_1,254258,0,0,14,0
4,2021-04-01,Prague_1,242598,4,1,0,0


In [7]:
data['ds'] = pd.to_datetime(data['ds'])

In [8]:
#Splitting the Data for Train & Test
train = data[data['ds'] <= '2023-07-01']
test = data[data['ds'] > '2023-07-01']

In [9]:
Y_test_df = test.copy()
df = train.copy()

In [11]:
tfm = timesfm.TimesFm(
    context_len=512,
    horizon_len=8,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

...

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 4.72 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 2.73 seconds.
Jitting decoding.
Jitted decoding in 41.70 seconds.


In [12]:
timesfm_df = tfm.forecast_on_df(
    inputs=df,
    freq='MS',
    value_name='y',
    num_jobs=-1,
)

Processing dataframe with multiple processes.


2024-09-12 04:56:24.037573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-09-12 04:56:24.037573: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64


Finished preprocessing dataframe.
Finished forecasting.


In [13]:
timesfm_df.to_csv("timesfm_monthly.csv")